# Project Capstone

## Import Libraries

In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import zipfile
import os

In [2]:
!pip install kaggle

In [2]:
!kaggle datasets download -d uom190346a/sleep-health-and-lifestyle-dataset

Dataset URL: https://www.kaggle.com/datasets/uom190346a/sleep-health-and-lifestyle-dataset
License(s): CC0-1.0
  0% 0.00/2.54k [00:00<?, ?B/s]
100% 2.54k/2.54k [00:00<00:00, 3.95MB/s]


In [3]:
with zipfile.ZipFile('sleep-health-and-lifestyle-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('data')
os.remove('sleep-health-and-lifestyle-dataset.zip')

## Data Wrangling

In [4]:
data = pd.read_csv("./data/Sleep_health_and_lifestyle_dataset.csv")
data = pd.DataFrame(data)
data.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Person ID                374 non-null    int64  
 1   Gender                   374 non-null    object 
 2   Age                      374 non-null    int64  
 3   Occupation               374 non-null    object 
 4   Sleep Duration           374 non-null    float64
 5   Quality of Sleep         374 non-null    int64  
 6   Physical Activity Level  374 non-null    int64  
 7   Stress Level             374 non-null    int64  
 8   BMI Category             374 non-null    object 
 9   Blood Pressure           374 non-null    object 
 10  Heart Rate               374 non-null    int64  
 11  Daily Steps              374 non-null    int64  
 12  Sleep Disorder           155 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 38.1+ KB


In [ ]:
data.isna().sum()

Person ID                    0
Gender                       0
Age                          0
Occupation                   0
Sleep Duration               0
Quality of Sleep             0
Physical Activity Level      0
Stress Level                 0
BMI Category                 0
Blood Pressure               0
Heart Rate                   0
Daily Steps                  0
Sleep Disorder             219
dtype: int64

In [ ]:
data.fillna(value="None", inplace=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Person ID                374 non-null    int64  
 1   Gender                   374 non-null    object 
 2   Age                      374 non-null    int64  
 3   Occupation               374 non-null    object 
 4   Sleep Duration           374 non-null    float64
 5   Quality of Sleep         374 non-null    int64  
 6   Physical Activity Level  374 non-null    int64  
 7   Stress Level             374 non-null    int64  
 8   BMI Category             374 non-null    object 
 9   Blood Pressure           374 non-null    object 
 10  Heart Rate               374 non-null    int64  
 11  Daily Steps              374 non-null    int64  
 12  Sleep Disorder           374 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 38.1+ KB


In [ ]:
print("Jumlah duplikasi: ", data.duplicated().sum())

Jumlah duplikasi:  0


In [ ]:
dataset = data.drop(columns=['Person ID', 'Occupation', 'Blood Pressure'])
dataset.head()

,Gender,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Heart Rate,Daily Steps,Sleep Disorder
0,Male,27,6.1,6,42,6,Overweight,77,4200,None
1,Male,28,6.2,6,60,8,Normal,75,10000,None
2,Male,28,6.2,6,60,8,Normal,75,10000,None
3,Male,28,5.9,4,30,8,Obese,85,3000,Sleep Apnea
4,Male,28,5.9,4,30,8,Obese,85,3000,Sleep Apnea


In [ ]:
dataset.describe()

,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,Heart Rate,Daily Steps
count,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000
mean,42.184492,7.132086,7.312834,59.171123,5.385027,70.165775,6816.844920
std,8.673133,0.795657,1.196956,20.830804,1.774526,4.135676,1617.915679
min,27.000000,5.800000,4.000000,30.000000,3.000000,65.000000,3000.000000
25%,35.250000,6.400000,6.000000,45.000000,4.000000,68.000000,5600.000000
50%,43.000000,7.200000,7.000000,60.000000,5.000000,70.000000,7000.000000
75%,50.000000,7.800000,8.000000,75.000000,7.000000,72.000000,8000.000000
max,59.000000,8.500000,9.000000,90.000000,8.000000,86.000000,10000.000000


## Data Encoding

In [ ]:
label_encoders = {}
for column in dataset.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    dataset[column] = label_encoders[column].fit_transform(dataset[column])

In [ ]:
dataset.describe()

## Split Data

In [ ]:
x = dataset.drop(columns=['Quality of Sleep'])
y = dataset[['Quality of Sleep']]

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state=42)

print("Training data size:", len(x_train))
print("Validation data size:", len(x_val))

print("\nTraining label size:", len(y_train))
print("Validation label size:", len(y_val))

In [ ]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)

## Machine Learning

In [ ]:
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs['mae'] <= 0.1 and logs['val_mae'] <= 0.1 and epoch >= 500:
            print("Early stopping triggered at epoch ", epoch, " because validation MAE is less than 0.1 and training MAE is less than 0.1 and epoch is greater than 500")
            self.model.stop_training = True

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(x_train.shape[1],)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mean_squared_error',
    metrics=['mae']
)

In [ ]:
history = model.fit(x_train_scaled, y_train,
                    epochs=1000,
                    validation_data=(x_val_scaled, y_val),
                    batch_size=32,
                    callbacks=[CustomCallback()]
)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
mae = history.history['mae']
val_mae = history.history['val_mae']

epochs = range(1, len(loss) + 1)

plt.figure(figsize=(14, 5))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(epochs, loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.legend()

# MAE plot
plt.subplot(1, 2, 2)
plt.plot(epochs, mae, label='Training MAE')
plt.plot(epochs, val_mae, label='Validation MAE')
plt.title('Training and Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('Mean Absolute Error')
plt.legend()

plt.show()

In [ ]:
print(x_val_scaled[0])

In [ ]:
predictions = model.predict(x_train_scaled)

print("Beberapa prediksi:")
for i in range(10):
    for j in range(y_train.shape[1]):
        feature_name = y_train.columns[j]
        rounded_pred = round(predictions[i][j])
        if feature_name in label_encoders:
            pred_original = label_encoders[feature_name].inverse_transform([rounded_pred])[0]
            actual_original = label_encoders[feature_name].inverse_transform([y_train.iloc[i, j]])[0]
        else:
            pred_original = predictions[i][j]
            actual_original = y_train.iloc[i, j]
        try:
            pred_original = float(pred_original)
            actual_original = float(actual_original)
            print(f"Prediksi untuk target {feature_name}: {pred_original:.2f}, Nilai aktual: {actual_original:.2f}")
        except ValueError:
            print(f"Prediksi untuk target {feature_name}: {pred_original}, Nilai aktual: {actual_original}")
    print()

In [ ]:
predictions = model.predict(x_val_scaled)

print("Beberapa prediksi pertama:")
for i in range(1):
    for j in range(y_train.shape[1]):
        print(f"Prediksi untuk target {y_val.columns[j]}: {predictions[i][j]:.2f}, Nilai aktual: {y_val.iloc[i, j]:.2f}")
    print()

In [ ]:
dataset.iloc[56]

Gender                       Male
Age                            32
Sleep Duration                7.7
Quality of Sleep                7
Physical Activity Level        75
Stress Level                    6
BMI Category               Normal
Heart Rate                     70
Daily Steps                  8000
Sleep Disorder               None
Name: 56, dtype: object

In [ ]:
# print hasil setelah di proses label encoder
print(dataset['BMI Category'][1])

## Save Model

In [ ]:
model.save('model_1_fix.h5')

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model_1_fix.tflite', 'wb') as f:
  f.write(tflite_model)

## Load Model

In [ ]:
loaded_model = tf.keras.models.load_model('model_1_fix.h5')

In [ ]:
loaded_model.summary()

In [ ]:
predictions = loaded_model.predict(x_train_scaled)

print("Beberapa prediksi:")
for i in range(1):
    for j in range(y_train.shape[1]):
        feature_name = y_train.columns[j]
        rounded_pred = round(predictions[i][j])
        if feature_name in label_encoders:
            pred_original = label_encoders[feature_name].inverse_transform([rounded_pred])[0]
            actual_original = label_encoders[feature_name].inverse_transform([y_train.iloc[i, j]])[0]
        else:
            pred_original = predictions[i][j]
            actual_original = y_train.iloc[i, j]
        try:
            pred_original = float(pred_original)
            actual_original = float(actual_original)
            print(f"Prediksi untuk target {feature_name}: {pred_original:.2f}, Nilai aktual: {actual_original:.2f}")
        except ValueError:
            print(f"Prediksi untuk target {feature_name}: {pred_original}, Nilai aktual: {actual_original}")
    print()

In [ ]:
predictions = loaded_model.predict(x_val_scaled)

print("Beberapa prediksi pertama:")
for i in range(1):
    for j in range(y_val.shape[1]):
        print(f"Prediksi untuk target {y_val.columns[j]}: {predictions[i][j]:.2f}, Nilai aktual: {y_val.iloc[i, j]:.2f}")
    print()